# Shifting science frames

Written by Evgenii N.

The following code shifts the science frames so that stars appear at same x-y positions in all images. The input data is in `030_science_frames/data/reduced` directory and the shifted images are saved to `040_shift/data/shifted`.

## Prerequisite code

In [ ]:
# Import libraries that we will use later in this notebook
import os
import shutil
import ccdproc
import numpy as np
from astropy.visualization import ZScaleInterval, MinMaxInterval, ImageNormalize
from astropy import units as u
from matplotlib.colors import LogNorm
from ccdproc import CCDData
import matplotlib.pyplot as plt

# Make images non-blurry on high pixel density screens
%config InlineBackend.figure_format = 'retina'

# Title size
plt.rcParams['axes.titlesize'] = 16

# Axes label size
plt.rcParams['axes.labelsize'] = 13


def show_image(image, title):
    """
    Display an image.
    
    Parameters
    ---------
    
    image: astropy.nddata.ccddata.CCDData
        A fits image to show.
        
    title: str
        Plot title.
    """
    fig, ax = plt.subplots(figsize=(12, 8))  # Change image size
    plt.rcParams.update({'font.size': 10})  # Change font size
    
    # Scale the image similar to 'zscale' mode in DS9.
    # This makes easier to spot things in the image.
    interval=ZScaleInterval()
    vmin, vmax = interval.get_limits(image)
    norm = ImageNormalize(vmin=vmin, vmax=vmax)
    
    plt.imshow(image, cmap='gray', norm=norm)  # Set color map and pixel scaling
    plt.xlabel('x [pixel]')  # Set axis labels
    plt.ylabel('y [pixel]')
    plt.title(title, y=-0.2)  # Set image title
    plt.colorbar()  # Show color bar
    

def print_image_stats(image, title):
    """
    Print first pixel value, average and standard deviation for an image.
    
    Parameters
    ---------
    
    image: astropy.nddata.ccddata.CCDData
        A fits image to show.
        
    title: str
        Image name.
    """
    
    data = np.asarray(image)# Get numpy array for image data
    label_len = 10  # Length of the text label
    first_pixel = data[0, 0]  # First pixel
    average = np.mean(data)  # Average
    standard_deviation = np.std(data)   # Standard deviation


    # Print values
    # -------

    print(
        f'\n{title}',
        f"\n{'-' * len(title)}",
        f"\n{'Pixel:':<10}{first_pixel:>10.2f} ADU",
        f"\n{'Avg:':<10}{average:>10.2f} ADU",
        f"\n{'Std:':<10}{standard_deviation:>10.2f} ADU\n"
    )
    

def save_image(image, file_path):
    """
    Save image to disk. Overwrites the file if it already exist.
    
    Parameters
    ---------
    image: astropy.nddata.ccddata.CCDData
        Image to be saved
        
    file_path: str
        Path where the image is saved
    """
    
    # Delete the file if it already exists
    
    try:
        os.remove(file_path)
    except OSError:
        pass
    
    # Create directory
    # ------

    dirname = os.path.dirname(file_path)
    
    if not os.path.exists(dirname):
        os.makedirs(dirname)

    image.write(file_path)

## Program code

In [ ]:
def reduce_science_for_filter(filter, reduced_path,
                              fits_path, bias, dark, flat, dark_temp):
    """
    Reduce the science files for the given filter.
    
    Parameters
    ----------
    
    filter: str
        Name of the filter: 'V', 'B' etc.
        
    reduced_path: str
        Path to directory where reduced science images are saved.
        
    fits_path: str
        Directory where science files are located.
        
    bias: astropy.nddata.ccddata.CCDData
        Reduced bias image.
        
    dark: astropy.nddata.ccddata.CCDData
        Reduced dark image.
        
    flat: astropy.nddata.ccddata.CCDData
        Reduced flat image.
        
    dark_temp: float
        Temperature (in degrees) of the CCD in the dark image
        
    Returns
    -------
    
    list of astropy.nddata.ccddata.CCDData
        Reduced science images
    """
    
    print(f"\nReducing science for {filter} filter:")
    
    # Get names of all image files in current directory
    files = ccdproc.ImageFileCollection(fits_path, glob_include = f'*_{filter}_*')
    files = files.files_filtered(PICTTYPE = 1)
        
    # Read the images and store them in a list
    # --------

    sci = [
            CCDData.read(os.path.join(fits_path, file_name), unit="adu")
            for file_name in files
          ]
    
    for file_data in zip(files, sci):
        ccd_temp = file_data[1].header["CCD-TEMP"]
        
        print_text = f'{file_data[0]} ccd_temp={ccd_temp:.2f} C'
        
        # Make sure science and bias temperatures are within 0.5 degrees
        if abs(ccd_temp - dark_temp) > 0.5:
            message = f'Large temperature difference between bias and science\n{print_text}'
            raise RuntimeError(message)
        
        print(print_text)
    
    
    print_image_stats(sci[0], title=f"First science {filter}")
    print(f"First science exposure time: {sci[0].header['EXPTIME']} s")

    # Subtract bias from science frame
    # --------
    
    sci_bias_subtracted = [
        ccdproc.subtract_bias(image, bias)
        for image in sci
    ]

    print_image_stats(sci_bias_subtracted[0], title='Science image, bias subtracted')
    
    # Subtract dark median image from flats.
    # We also scale by exposure time to make sure
    # the effective exposures of two images are equal.
    # --------

    sci_bias_median_subtracted = [
        ccdproc.subtract_dark(image, dark, exposure_time='EXPTIME', 
                              exposure_unit=u.second, scale=True)
        for image in sci_bias_subtracted
    ]

    print_image_stats(sci_bias_median_subtracted[0], title='Science image, dark subtracted')

    # Divide by flat
    # ---------

    sci_bias_median_subtracted_flat_corrected = [
        ccdproc.flat_correct(image, flat)
        for image in sci_bias_median_subtracted
    ]

    print_image_stats(sci_bias_median_subtracted_flat_corrected[0], title='Science image, flat corrected')
    
    # Save reduced science images to disk
    # -------
    
    for reduced, file_name in zip(sci_bias_median_subtracted_flat_corrected, files):
        file_path = os.path.join(reduced_path, file_name)
        save_image(image=reduced, file_path=file_path)

In [ ]:
# Set Bias and Dark image paths
# -------

data_dir = './data/'
dark_dir = '../010_bias_and_dark'
flat_dir = '../020_flat/data/flats/reduced'
bias_path = os.path.join(dark_dir, 'bias_median.fits')
dark_path = os.path.join(dark_dir, 'dark_median.fits')

# Load bias
bias = CCDData.read(bias_path, unit="adu")
show_image(bias, title='Figure 55: Reduced bias')
print_image_stats(bias, title="Bias")
print(f"Bias exposure time: {bias.header['EXPTIME']} s")

# Load dark
dark = CCDData.read(dark_path, unit="adu")
show_image(dark, title='Figure 56: Reduced dark')
print_image_stats(dark, title="Dark")
print(f"Dark exposure time: {dark.header['EXPTIME']} s")

# Reduce images
# -------

nights = ['march_09_2018', 'march_29_2018', 'april_30_2018']

# Reduce images for all nights
for night in nights:
    filters = ['B', "I", "R", "V"]

    # Reduce science frames for all filters
    for i, filter in enumerate(filters):
        # Load flat image
        flat_path = os.path.join(flat_dir, f'flat_{filter}_median.fits')
        flat = CCDData.read(flat_path, unit="adu")

        fits_path = os.path.join(data_dir, f'science_unreduced/{night}')

        sci = reduce_science_for_filter(
            filter=filter,
            reduced_path=os.path.join(data_dir, f'reduced/{night}'),
            fits_path=fits_path,
            bias=bias, dark=dark, flat=flat,
            dark_temp=-5.23)
        
print("------------------")
print("We are done")

## Sanity checks for March 9 images

* The average pixel value of raw science V-band image is about 243 ADU.
* Subtracting bias reduces it to 134 ADU, which is expected, since bias is about 110 ADU.
* Subtracting data reduces it to 133 ADU, which is expected, since dark is 10 adu with exposure 10 times longer than science exposure.
* Dividing by flat does not change the average ADU, which is expected, since flat has average pixel value of 1.
* The average pixel values for images from March 29 and April 30 also look reasonable.

### Science image before and after reduction

A science frame before and after reduction is shown on Fig. 58. I can see that reduction worked because:

* There are fewer hot pixels in reduced image, they were removed after dark subtraction, there are some remaining hot pixels left, which will hopefully be removed when we combine the frames,

* The left side of the raw image has bright glow, which is removed in reduced image after bias subtraction.

* I then manually compare the raw and reduced images for about thirty other frames, found nothing weird, effect of reduction on other frames look like Fig. 58.

![Science](images/sci_before_after.jpg)
Figure 58: Frame `NGC_3201_B_60.000secs_00001604` form March 9 archive before (left) and after (right) reduction. Reduced image has smaller number of hot pixels and not glow in the left side.

I'm ALMOST happy with the reduction code. We still have some hot pixels left in reduced image (right image in Fig. 58). Vaishali said in private email that it's ok to have some hot pixels remaining, we just need to make sure not to measure fluxes from stars that overlap with those hot pixels.